<a href="https://colab.research.google.com/github/focusdivyam/focusdivyam/blob/main/cross_language_Writer_Identification_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Style-focused Features:

# Local features: captures basic stroke properties
# Style features: focuses on texture and writing patterns
# Stroke features: analyzes stroke formation and connections


# Language-Invariant Design:

# Multi-scale attention to focus on style rather than character shapes
# Global style representation through feature aggregation
# Emphasis on stroke patterns and writing dynamics

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

class StyleEncoder(nn.Module):
    def __init__(self):
        super(StyleEncoder, self).__init__()

        # Local feature extraction
        self.local_features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        # Texture and style features
        self.style_features = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        # Stroke pattern analysis
        self.stroke_features = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

    def forward(self, x):
        # Extract different levels of features
        local_feat = self.local_features(x)
        style_feat = self.style_features(local_feat)
        stroke_feat = self.stroke_features(style_feat)

        return local_feat, style_feat, stroke_feat

class TeacherNetwork(nn.Module):
    def __init__(self, num_writers):
        super(TeacherNetwork, self).__init__()

        # Style encoder
        self.style_encoder = StyleEncoder()

        # Multi-scale feature aggregation
        self.attention1 = nn.Sequential(
            nn.Conv2d(64, 1, kernel_size=1),
            nn.Sigmoid()
        )
        self.attention2 = nn.Sequential(
            nn.Conv2d(128, 1, kernel_size=1),
            nn.Sigmoid()
        )
        self.attention3 = nn.Sequential(
            nn.Conv2d(256, 1, kernel_size=1),
            nn.Sigmoid()
        )

        # Global average pooling
        self.gap = nn.AdaptiveAvgPool2d(1)

        # Style classifier
        self.classifier = nn.Sequential(
            nn.Linear(64 + 128 + 256, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_writers)
        )

    def forward(self, x):
        # Extract multi-level style features
        local_feat, style_feat, stroke_feat = self.style_encoder(x)

        # Apply attention to each level
        local_att = self.attention1(local_feat)
        style_att = self.attention2(style_feat)
        stroke_att = self.attention3(stroke_feat)

        # Weighted features
        local_weighted = local_feat * local_att
        style_weighted = style_feat * style_att
        stroke_weighted = stroke_feat * stroke_att

        # Global pooling
        local_pool = self.gap(local_weighted).squeeze(-1).squeeze(-1)
        style_pool = self.gap(style_weighted).squeeze(-1).squeeze(-1)
        stroke_pool = self.gap(stroke_weighted).squeeze(-1).squeeze(-1)

        # Concatenate features
        combined_features = torch.cat([local_pool, style_pool, stroke_pool], dim=1)

        # Classification
        output = self.classifier(combined_features)

        return output, combined_features

class StudentNetwork(nn.Module):
    def __init__(self, num_writers):
        super(StudentNetwork, self).__init__()

        # Lightweight style encoder
        self.style_encoder = nn.Sequential(
            # Local and texture features
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            # Stroke patterns
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        # Global average pooling
        self.gap = nn.AdaptiveAvgPool2d(1)

        # Style classifier
        self.classifier = nn.Sequential(
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_writers)
        )

    def forward(self, x):
        # Extract style features
        features = self.style_encoder(x)
        # Global pooling
        pooled = self.gap(features).squeeze(-1).squeeze(-1)
        # Classification
        output = self.classifier(pooled)

        return output, pooled

class CrossLanguageWriterID:
    def __init__(self, num_writers):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.teacher = TeacherNetwork(num_writers).to(self.device)
        self.student = StudentNetwork(num_writers).to(self.device)

        # Loss functions
        self.classification_criterion = nn.CrossEntropyLoss()
        self.distillation_criterion = nn.MSELoss()

        # Preprocessing with additional augmentations
        self.transforms = transforms.Compose([
            transforms.Grayscale(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,)),
        ])

    def train_step(self, images, labels, temperature=2.0, alpha=0.5):
        # Get teacher predictions
        teacher_logits, teacher_features = self.teacher(images)
        student_logits, student_features = self.student(images)

        # Soft targets with temperature scaling
        soft_targets = F.softmax(teacher_logits / temperature, dim=1)
        student_soft = F.softmax(student_logits / temperature, dim=1)

        teacher_features = teacher_features[:, :student_features.shape[1]]
        ########### added (size-ntmatch)

        # Compute losses
        hard_loss = self.classification_criterion(student_logits, labels)
        soft_loss = self.distillation_criterion(student_soft, soft_targets.detach())
        feature_loss = self.distillation_criterion(student_features, teacher_features.detach())

        # Combined loss with feature matching
        total_loss = (alpha * hard_loss +
                     (1 - alpha) * temperature * temperature * soft_loss +
                     0.1 * feature_loss)

        return total_loss

    def identify_writer(self, image):
        image = self.transforms(image).unsqueeze(0).to(self.device)

        with torch.no_grad():
            logits, _ = self.student(image)
            probabilities = F.softmax(logits, dim=1)
            predicted_writer = torch.argmax(probabilities).item()
            confidence = probabilities[0][predicted_writer].item()

        return predicted_writer, confidence

In [3]:
import os
from torchvision.datasets import DatasetFolder
from PIL import Image

class WriterDataset(DatasetFolder):
    def __init__(self, root_dir, transform=None):
        super().__init__(root=root_dir, loader=self._image_loader, extensions=("tif", "jpg", "jpeg"))
        self.transform = transform

    @staticmethod
    def _image_loader(path):
        return Image.open(path).convert("L")

    def __getitem__(self, index):
        path, label = self.samples[index]
        sample = self.loader(path)
        if self.transform:
            sample = self.transform(sample)
        return sample, label


In [6]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [1]:
from torch.utils.data import DataLoader, random_split

# Dataset path
dataset_path = dataset_path = "/content/drive/My Drive/Eng"

# Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

# Create dataset and split into train/test sets
dataset = WriterDataset(dataset_path, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


NameError: name 'transforms' is not defined

In [ ]:
import torch.optim as optim

def train_model(model, train_loader, num_epochs=10, learning_rate=0.001):
    # Move the teacher and student networks to the device
    model.teacher.to(model.device)
    model.student.to(model.device)

    optimizer = optim.Adam(model.student.parameters(), lr=learning_rate)
    model.student.train()


    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(model.device), labels.to(model.device)

            optimizer.zero_grad()
            loss = model.train_step(images, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss / len(train_loader):.4f}")

In [ ]:
def test_model(model, test_loader):
    model.student.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(model.device), labels.to(model.device)
            logits, _ = model.student(images)
            _, predicted = torch.max(logits, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    print(f"Test Accuracy: {accuracy * 100:.2f}%")


In [ ]:
# Initialize model
num_writers = 10  # Replace with the actual number of writers in your dataset
model = CrossLanguageWriterID(num_writers)

# Train the model
train_model(model, train_loader, num_epochs=20, learning_rate=0.001)

# Test the model
test_model(model, test_loader)


KeyboardInterrupt: 